In [1]:

import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import *

In [3]:
from pyspark.ml.feature  import Tokenizer,RegexTokenizer,StopWordsRemover,CountVectorizer,OneHotEncoderEstimator,StringIndexer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression

In [4]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml.feature import Word2Vec
import pyspark.sql.functions as f
from pyspark.ml.feature import Word2VecModel

In [5]:
sc=pyspark.SparkContext()

In [6]:
spark= SparkSession \
       .builder \
       .appName("DICPA3") \
       .config("spark.some.config.option","some-value") \
       .getOrCreate() 

In [7]:
train_pd = pd.read_csv('train.csv')

In [8]:
mapping_pd = pd.read_csv('mapping.csv')


In [9]:
mapping=spark.createDataFrame(mapping_pd)


In [10]:
train=spark.createDataFrame(train_pd)

In [11]:
train.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [12]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")


train=regexTokenizer.transform(train)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

train=stopwordsRemover.transform(train)

# word2Vectrain = Word2Vec(vectorSize=150, inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltrain = word2Vectrain.fit(train)
# modeltrain.save('./part3trainw2v')




In [13]:

trainmodel = Word2VecModel.load('./part3trainw2v')
train = trainmodel.transform(train)

In [14]:
mapping=spark.createDataFrame(mapping_pd,['id',"genre"])
mapgenre=mapping.select("genre","id").rdd.collectAsMap()

In [15]:
mapping.show()
count=mapping.count()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [16]:

train.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[-0.0567182510242...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.03953355165605...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[0.00612506371926...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|[0.01885360926169...|
|  595909|   A Cry in the Dark|Seventh-da

In [17]:

def function1(g):
    x=[]
    genre=g[1:-1].split(",")  
    for i in genre:
        value=i.strip()[1:-1]
        x.append(mapgenre.get(value))
    x.sort()
    return x
myfunction1=f.udf(function1,ArrayType(IntegerType()))
train=train.withColumn("index",myfunction1("genre"))

train.select('index').show()

+----------------+
|           index|
+----------------+
|          [0, 5]|
|  [0, 4, 10, 17]|
|      [0, 4, 16]|
|             [1]|
|       [0, 5, 6]|
|   [0, 3, 4, 10]|
|       [0, 3, 7]|
|             [0]|
|[0, 1, 2, 8, 19]|
|    [12, 13, 15]|
|             [1]|
|    [0, 1, 5, 6]|
|             [1]|
|             [1]|
|             [7]|
|   [3, 6, 7, 18]|
|             [0]|
| [2, 3, 4, 6, 9]|
|          [0, 9]|
|             [0]|
+----------------+
only showing top 20 rows



In [18]:

def function2(g):
    y=1
    x=[]
    for i in range(0,count):
        x.append(0)
    
    
    for i in g:
        x[i]=1
        
    
    return x
myfunction2=f.udf(function2,ArrayType(IntegerType()))
traindata=train.withColumn("label",myfunction2("index"))
traindata.select('features','index','label').show()

+--------------------+----------------+--------------------+
|            features|           index|               label|
+--------------------+----------------+--------------------+
|[-0.0567182510242...|          [0, 5]|[1, 0, 0, 0, 0, 1...|
|[0.03953355165605...|  [0, 4, 10, 17]|[1, 0, 0, 0, 1, 0...|
|[0.00612506371926...|      [0, 4, 16]|[1, 0, 0, 0, 1, 0...|
|[0.01885360926169...|             [1]|[0, 1, 0, 0, 0, 0...|
|[-0.0117615198380...|       [0, 5, 6]|[1, 0, 0, 0, 0, 1...|
|[0.04305206441959...|   [0, 3, 4, 10]|[1, 0, 0, 1, 1, 0...|
|[6.46465915596240...|       [0, 3, 7]|[1, 0, 0, 1, 0, 0...|
|[-0.0274835892556...|             [0]|[1, 0, 0, 0, 0, 0...|
|[0.00870116330497...|[0, 1, 2, 8, 19]|[1, 1, 1, 0, 0, 0...|
|[0.03890713853998...|    [12, 13, 15]|[0, 0, 0, 0, 0, 0...|
|[-0.0026940880472...|             [1]|[0, 1, 0, 0, 0, 0...|
|[0.01413125867045...|    [0, 1, 5, 6]|[1, 1, 0, 0, 0, 1...|
|[-0.0109163611606...|             [1]|[0, 1, 0, 0, 0, 0...|
|[-0.0089385102076...|  

In [19]:
test_pd = pd.read_csv('test.csv')
test=spark.createDataFrame(test_pd)
test.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [20]:



test=regexTokenizer.transform(test)



test=stopwordsRemover.transform(test)





In [21]:
# word2Vectest = Word2Vec( vectorSize=150,inputCol="filtered", outputCol="features" ,maxIter=1)

# modeltest= word2Vectest.fit(test)
# modeltest.save('./part3testw2v')



In [22]:
testmodel = Word2VecModel.load('./part3testw2v')

testdata = testmodel.transform(test)
testdata.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[-0.0117688054873...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[-0.0423577698182...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[-0.0203951604945...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|[-0.0382373130505...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier, of, ...|[soldier, fortune...|[-0.0054765218155...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...

In [27]:
x=[]

for i in range(0,count):
    
#     def parsePoint(line):
    
#         return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
#     parsedData = traindata.rdd.map(parsePoint)
#     part3model=LogisticRegressionWithLBFGS.train(parsedData)
    filename='./part3test'+str(i)
#    part3model.save(spark,filename)
    newmodel = LogisticRegressionModel.load(spark,filename)
    labelsAndPreds = testdata.rdd.map(lambda p: (p.movie_id, newmodel.predict(MLLibVectors.fromML(p.features))))
    x.append(labelsAndPreds.collect())

In [28]:
y={}
for i in x:
    for j in i:
        if j[0] in y:
            y[j[0]].append(j[1])
        else:
            y[j[0]]=[]
            y[j[0]].append(j[1])
for i in y:
    print(i,y[i])

1335380 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29062594 [0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
9252321 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
13455076 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
24165951 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1925869 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
10799612 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
28238240 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
17124781 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
28207941 [1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]
19174305 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
18392317 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
34420857 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
4039635 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
8034072 [1, 1, 1, 0, 1, 

20581421 [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
14195429 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
5604840 [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
1644429 [1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
23819161 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2926860 [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
34613355 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
26428662 [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
12605892 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
19746061 [1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]
6143174 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
4881121 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
3868207 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
32817768 [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
9215002 [0, 1, 1, 0, 0, 0,

26195831 [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
4066047 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
17595302 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
24236496 [1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]
3347962 [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
35271106 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
289890 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1250208 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
16843109 [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
31775043 [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
23837735 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
34269989 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
5118135 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
1662223 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
9222167 [1, 1, 1, 0, 1, 0, 

13544172 [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
10977 [0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
2360326 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
31843970 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
25387001 [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
5612517 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
17548651 [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
677920 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
9930507 [1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
22895245 [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
2025985 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
2567937 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
1346925 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2184458 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
9136345 [0, 1, 0, 1, 0, 1, 1, 1,

29728646 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
8435438 [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
2066226 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
22558992 [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2294801 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
9234454 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
436955 [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
1934924 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
31572779 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
468356 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
24473950 [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
14454173 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
20533851 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
24998748 [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1]
29836913 [0, 1, 1, 0, 1, 0, 0

25619875 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
13911153 [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
17139347 [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24542123 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
8250603 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
12271243 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
180993 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
30092412 [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
4605171 [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
17719953 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
14330111 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
5336170 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
10116734 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
15293788 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
19186528 [0, 0, 0, 1, 0, 

9752270 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
1771007 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
30323966 [0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
20814932 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1]
17523604 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
33059372 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
15323456 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
12794862 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
6631279 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
1365735 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
25725663 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
8808797 [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
28221667 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
32069237 [1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]
25433930 [0, 1, 0, 1, 0, 

31631116 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
20312589 [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2175063 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
23998788 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
2337131 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
31139138 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
15185206 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
19571819 [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
10104497 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
2253183 [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
14489554 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
5170537 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
1781048 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
4680482 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
1524739 [0, 1, 1, 0, 1, 0,

1273558 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
5273926 [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
2304640 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
10640368 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
23978412 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
33159887 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
6840273 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
8039067 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
21404478 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
3303352 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
32956666 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
17510112 [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]
1714944 [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
526277 [0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]
20398475 [0, 1, 1, 0, 0, 0, 0

17228730 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
7277626 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
27100903 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
16833930 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
10404370 [1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
35189883 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
11564856 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
24490679 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
4644114 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
30136288 [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
31913252 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
11396944 [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
27548546 [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]
26695603 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
24549492 [0, 1, 0, 1, 

In [29]:
keypred={}

for i in y:
    x=""
    for j in y[i]:
        x=x+" "+str(j)
    x=x.strip()
    keypred[i]=x
for i in keypred:
    print(i,str(keypred[i]))

1335380 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
29062594 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 1
9252321 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
13455076 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1
24165951 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1925869 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1
10799612 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
28238240 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
17124781 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0
28207941 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1
19174305 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
18392317 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
34420857 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
4039635 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
8034072 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
4016437 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
1520023 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
24589422 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
35068740 0 1 1 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1
21132951 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
6575053 0 0 0 1 0 1 1 1 0 0

4698619 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
7274197 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
2709646 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1
29197597 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
34414967 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
7208117 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1
18803418 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1
5020303 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
583273 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
4584458 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
3139643 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
3945952 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1
32455629 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
32638698 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
1339563 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
32694962 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1
14611324 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1
534408 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
15553721 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
15352210 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0
14613035 0 0 0 1 1 1 1 0 0 0 0 0 

2618462 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0
7482786 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
18878308 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
26693888 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1
5321761 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
5565988 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1
24644162 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
19615593 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0
12607294 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 1
20768445 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1
745100 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 1 1
19768754 0 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1
4091931 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
9566099 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
34952682 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1
31197541 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1
458890 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
1586160 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
31453213 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
26091760 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
21369620 1 1 1 0 1 0 0 0 1 0 0 

16047670 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0
4611613 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
32972617 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
14811847 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
10107470 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
3342195 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
20833484 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
13775502 0 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1
31743981 0 1 1 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1
236283 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
15703732 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
25271596 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
33099484 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
2541685 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
1121941 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
8982899 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
19662785 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
12761104 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1
31332767 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
261402 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1
3918828 0 1 0 0 1 0 0 0 0 0 0

16195082 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
24665513 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1
33152192 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
28103735 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
7302636 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
25098571 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
31630826 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1
11891977 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
36061742 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
91133 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1
19233962 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
3304471 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
9807797 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0
1232511 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0
28150094 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
16725177 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
26212916 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
8441185 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
24463571 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 1
527169 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
590598 0 1 0 0 0 0 0 1 0 0 0 0

8988995 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
11445288 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0
19353888 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1
23448467 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
30574080 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
6173832 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0
5735467 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
666948 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
8050436 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1
31824067 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
25159111 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1
15688731 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1
5877549 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
19898489 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
4907569 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0
16586081 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
24467898 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
2903396 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
3383952 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1
1722134 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11021037 0 1 1 0 1 0 0 0 1 0 0 

30403926 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
2974485 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
21423308 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
20090822 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1
6745617 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0
12569581 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1
967721 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
6283540 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
29271529 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
5512801 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1
32600481 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0
4541896 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
73170 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1
525292 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
9399204 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
8909666 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
23624423 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
1546168 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1
32099247 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
23767889 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
11876523 0 1 1 0 0 0 1 0 1 0 0 0 0 

21534495 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
4399374 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0
12867093 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0
24236812 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
27231806 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
3493786 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1
25539481 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
1801710 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
28256111 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1
32112949 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0
12595048 0 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 1
5655933 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
15607591 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
36000792 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0
12004855 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
12481201 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1
35912502 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
12556506 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
24519551 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
1022323 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1
17062684 1 1 1 0 1 0 0 0 

113449 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
34378954 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0
4151108 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1
2291440 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
19637784 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
1151458 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
8307454 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1
4556313 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0
19366458 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
26298466 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
3324212 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
35460624 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
444061 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
3940904 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
2506530 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
8974603 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
34394367 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
15974702 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1
1064199 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
10572944 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1
14537381 0 1 1 0 0 1 0 0 0 0 0 0 0

7730324 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
7455727 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
982199 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
536521 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
910759 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
36372501 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1
1745339 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1
9437838 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
35591024 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 1
27997093 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
6168146 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0
3317623 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
33002536 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
25431279 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
18813698 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
1809414 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
4413498 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
15048916 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1
17806123 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
17395300 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1
29222508 0 1 0 0 0 0 0 0 0 0 0 0 0

In [30]:
import csv

with open ('output3.csv','w',newline="") as mfile:
    writer=csv.writer(mfile)
    writer.writerow(["movie_id","predictions"])
    for key , value in keypred.items():
        writer.writerow([key,value])